In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
import pandas as pd

emo_df = pd.read_csv('recovery-news-data-emotion-features.csv')
read_df=pd.read_csv('recovery-news-data-readability-features.csv')
write_df=pd.read_csv('recovery-news-data-writing-features.csv')

In [3]:
emo_df

,news_id,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,afraid,amused,angry,annoyed,dont_care,happy,inspired,sad,reliability
0,0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,0.187126,0.052152,0.058249,0.034350,0.284639,0.254297,0.067645,0.061542,1.0
1,1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,0.145114,0.064628,0.118878,0.041245,0.333777,0.174415,0.053452,0.068490,1.0
2,2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,0.216482,0.061477,0.063863,0.034224,0.198211,0.247231,0.080797,0.097716,1.0
3,3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,0.218814,0.044995,0.075313,0.033225,0.268769,0.223028,0.056909,0.078947,0.0
4,4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,0.205156,0.049457,0.065705,0.022823,0.288402,0.219217,0.060947,0.088293,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,0.109309,0.045679,0.058745,0.033031,0.297066,0.308920,0.076011,0.071239,0.0
2025,2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,0.070619,0.062367,0.056961,0.064059,0.352311,0.243119,0.085429,0.065135,0.0
2026,2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,0.105368,0.044128,0.044088,0.078532,0.247275,0.357812,0.083971,0.038825,0.0
2027,2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,0.063775,0.134295,0.095691,0.039837,0.334285,0.216420,0.078294,0.037404,0.0


In [4]:
read_df

,news_id,smog_index,flesch_reading_ease,flesch_kincaid_grade_level,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count
0,0,87.283108,-1004.215753,412.189204,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0
1,1,53.258245,-439.791414,199.088805,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0
2,2,79.773634,-1315.626760,535.393607,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0
3,3,75.390260,-709.048179,297.476011,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0
4,4,24.504239,-54.545175,48.950702,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,68.264429,-757.762706,320.696728,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0
2025,2025,42.293715,-247.947778,125.877778,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0
2026,2026,62.222207,-861.102992,361.450903,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0
2027,2027,57.923837,-374.876154,168.633173,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0


In [5]:
write_df

,news_id,num_nouns,num_propernouns,num_personalnouns,num_ppssessivenouns,num_whpronoun,num_determinants,num_whdeterminants,num_cnum,num_adverb,num_interjections,num_verb,num_adj,num_vbd,num_vbg,num_vbn,num_vbp,num_vbz
0,0,288.0,153.0,34.0,15.0,13.0,168.0,16.0,23.0,66.0,0.0,88.0,153.0,43.0,41.0,59.0,68.0,48.0
1,1,132.0,122.0,24.0,6.0,3.0,74.0,5.0,18.0,44.0,0.0,23.0,65.0,34.0,18.0,32.0,28.0,24.0
2,2,392.0,143.0,63.0,14.0,15.0,269.0,18.0,43.0,118.0,0.0,127.0,198.0,62.0,47.0,46.0,94.0,67.0
3,3,204.0,119.0,22.0,14.0,4.0,129.0,7.0,15.0,43.0,0.0,55.0,108.0,47.0,41.0,51.0,37.0,29.0
4,4,31.0,14.0,1.0,1.0,0.0,23.0,0.0,3.0,6.0,0.0,6.0,13.0,4.0,6.0,5.0,10.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,208.0,192.0,30.0,16.0,3.0,113.0,6.0,24.0,48.0,0.0,57.0,104.0,47.0,30.0,25.0,34.0,33.0
2025,2025,98.0,69.0,24.0,12.0,2.0,51.0,2.0,7.0,26.0,0.0,15.0,37.0,27.0,20.0,12.0,19.0,16.0
2026,2026,271.0,133.0,39.0,7.0,5.0,131.0,12.0,25.0,62.0,0.0,104.0,111.0,41.0,46.0,26.0,50.0,40.0
2027,2027,120.0,53.0,15.0,4.0,3.0,57.0,8.0,12.0,21.0,0.0,31.0,72.0,21.0,22.0,16.0,19.0,25.0


In [6]:
result = pd.concat([write_df, read_df], axis=1)
result

,news_id,num_nouns,num_propernouns,num_personalnouns,num_ppssessivenouns,num_whpronoun,num_determinants,num_whdeterminants,num_cnum,num_adverb,...,flesch_kincaid_grade_level,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count
0,0,288.0,153.0,34.0,15.0,13.0,168.0,16.0,23.0,66.0,...,412.189204,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0
1,1,132.0,122.0,24.0,6.0,3.0,74.0,5.0,18.0,44.0,...,199.088805,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0
2,2,392.0,143.0,63.0,14.0,15.0,269.0,18.0,43.0,118.0,...,535.393607,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0
3,3,204.0,119.0,22.0,14.0,4.0,129.0,7.0,15.0,43.0,...,297.476011,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0
4,4,31.0,14.0,1.0,1.0,0.0,23.0,0.0,3.0,6.0,...,48.950702,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,208.0,192.0,30.0,16.0,3.0,113.0,6.0,24.0,48.0,...,320.696728,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0
2025,2025,98.0,69.0,24.0,12.0,2.0,51.0,2.0,7.0,26.0,...,125.877778,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0
2026,2026,271.0,133.0,39.0,7.0,5.0,131.0,12.0,25.0,62.0,...,361.450903,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0
2027,2027,120.0,53.0,15.0,4.0,3.0,57.0,8.0,12.0,21.0,...,168.633173,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0


In [7]:
result=result[['num_nouns', 'num_propernouns', 'num_personalnouns',
       'num_ppssessivenouns', 'num_whpronoun', 'num_determinants',
       'num_whdeterminants', 'num_cnum', 'num_adverb', 'num_interjections',
       'num_verb', 'num_adj', 'num_vbd', 'num_vbg', 'num_vbn', 'num_vbp',
       'num_vbz', 'smog_index', 'flesch_reading_ease',
       'flesch_kincaid_grade_level', 'coleman_liau_index', 'gunning_fog_index',
       'ari_index', 'lix_index', 'dale_chall_score',
       'dale_chall_known_fraction', 'syllable_count', 'lexicon_count',
       'sentence_count']]

In [8]:
final_result=pd.concat([emo_df, result], axis=1)

In [9]:
final_result

,news_id,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,...,flesch_kincaid_grade_level,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count
0,0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,...,412.189204,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0
1,1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,...,199.088805,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0
2,2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,...,535.393607,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0
3,3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,...,297.476011,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0
4,4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,...,48.950702,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,...,320.696728,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0
2025,2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,...,125.877778,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0
2026,2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,...,361.450903,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0
2027,2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,...,168.633173,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0


In [10]:
final_result=final_result[['news_id', 'Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness',
       'Surprise', 'Trust', 'Objective', 'afraid', 'amused', 'angry',
       'annoyed', 'dont_care', 'happy', 'inspired', 'sad',
       'num_nouns', 'num_propernouns', 'num_personalnouns',
       'num_ppssessivenouns', 'num_whpronoun', 'num_determinants',
       'num_whdeterminants', 'num_cnum', 'num_adverb', 'num_interjections',
       'num_verb', 'num_adj', 'num_vbd', 'num_vbg', 'num_vbn', 'num_vbp',
       'num_vbz', 'smog_index', 'flesch_reading_ease',
       'flesch_kincaid_grade_level', 'coleman_liau_index', 'gunning_fog_index',
       'ari_index', 'lix_index', 'dale_chall_score',
       'dale_chall_known_fraction', 'syllable_count', 'lexicon_count',
       'sentence_count', 'reliability']]

In [11]:
final_result

,news_id,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,...,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count,reliability
0,0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,...,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0,1.0
1,1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,...,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0,1.0
2,2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,...,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0,1.0
3,3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,...,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0,0.0
4,4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,...,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,...,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0,0.0
2025,2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,...,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0,0.0
2026,2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,...,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0,0.0
2027,2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,...,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0,0.0


In [12]:
df_final=final_result.drop(columns=['news_id','reliability'],axis=1)

In [13]:
df_final

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,afraid,...,flesch_kincaid_grade_level,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count
0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,0.187126,...,412.189204,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0
1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,0.145114,...,199.088805,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0
2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,0.216482,...,535.393607,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0
3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,0.218814,...,297.476011,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0
4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,0.205156,...,48.950702,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,0.109309,...,320.696728,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0
2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,0.070619,...,125.877778,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0
2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,0.105368,...,361.450903,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0
2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,0.063775,...,168.633173,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0


In [14]:
X = df_final.iloc[:, 0:]


In [15]:
X

,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,afraid,...,flesch_kincaid_grade_level,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count
0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,0.187126,...,412.189204,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0
1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,0.145114,...,199.088805,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0
2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,0.216482,...,535.393607,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0
3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,0.218814,...,297.476011,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0
4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,0.205156,...,48.950702,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,0.109309,...,320.696728,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0
2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,0.070619,...,125.877778,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0
2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,0.105368,...,361.450903,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0
2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,0.063775,...,168.633173,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0


In [16]:
final_result['reliability']=final_result['reliability'].apply(lambda x:int(x))

In [17]:
final_result

,news_id,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust,Objective,...,coleman_liau_index,gunning_fog_index,ari_index,lix_index,dale_chall_score,dale_chall_known_fraction,syllable_count,lexicon_count,sentence_count,reliability
0,0,0.011846,0.040958,0.031052,0.061233,0.023925,0.039174,0.004960,0.068307,0.718545,...,21.736222,425.522148,530.109233,1087.007670,60.861630,0.421860,1990.0,1042.0,1.0,1
1,1,0.011874,0.027143,0.017349,0.041963,0.023242,0.037897,0.009701,0.059647,0.771184,...,18.940518,206.935458,257.398406,535.067729,33.517646,0.454183,877.0,502.0,1.0,1
2,2,0.018336,0.037914,0.046958,0.066391,0.030701,0.065443,0.012957,0.055853,0.665447,...,19.220051,550.878592,688.591738,1399.777126,75.341028,0.513196,2351.0,1364.0,1.0,1
3,3,0.012988,0.045399,0.021112,0.063752,0.031115,0.032674,0.004902,0.081468,0.706590,...,22.622249,307.367604,382.826867,794.121821,46.922592,0.374833,1474.0,747.0,1.0,0
4,4,0.026660,0.033526,0.042772,0.089552,0.020359,0.068824,0.005095,0.039643,0.673569,...,22.738509,50.512281,66.598158,159.614035,14.380453,0.447368,214.0,114.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2024,0.014073,0.053275,0.016001,0.034158,0.059056,0.027101,0.010837,0.079075,0.706424,...,19.720443,331.596064,413.503801,852.237392,48.676215,0.471095,1439.0,813.0,1.0,0
2025,2025,0.030234,0.034679,0.008621,0.057117,0.039077,0.035642,0.021987,0.067945,0.704699,...,19.052095,131.968254,164.016000,350.555556,23.594190,0.495238,519.0,315.0,1.0,0
2026,2026,0.012072,0.102877,0.005359,0.023160,0.021470,0.025328,0.015776,0.116851,0.677108,...,19.199717,372.257236,466.083993,953.167573,53.520354,0.497280,1623.0,919.0,1.0,0
2027,2027,0.017304,0.020901,0.007770,0.023442,0.013270,0.013475,0.006283,0.087273,0.810281,...,22.766851,175.246154,217.468053,460.951923,30.654177,0.365385,847.0,416.0,1.0,0


In [18]:
y = final_result.iloc[:,-1]
y

0       1
1       1
2       1
3       0
4       1
       ..
2024    0
2025    0
2026    0
2027    0
2028    0
Name: reliability, Length: 2029, dtype: int64

In [19]:
y=np.array([x for x in y])

In [20]:
y

array([1, 1, 1, ..., 0, 0, 0])

# Xgboost

In [34]:

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from xgboost import XGBClassifier


avgP=0
avgAUROC=0
cv=5
skf = StratifiedKFold(n_splits=cv,random_state=10,shuffle=True)
for train_index, val_index in skf.split(X.values, y):
  X_train1=X.values[train_index]
  X_val=X.values[val_index]
  y_train1=y[train_index]
  y_val=y[val_index]
  clf=XGBClassifier(objective="binary:logistic", random_state=100)
  clf.fit(X_train1,y_train1)
  y_pred_val=clf.predict_proba(X_val)[:,1]
  #print(y_pred_val)
  avgP+=average_precision_score(y_val, y_pred_val)
  #print(average_precision_score(y_val, y_pred_val))
  avgAUROC+=roc_auc_score(y_val, y_pred_val)
  #print(roc_auc_score(y_val, y_pred_val))
avgP=avgP/cv
avgROC=avgAUROC/cv
print('AUROC: ',avgROC)
print('Avg Precision: ',avgP)

C:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:43] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:44] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[15:10:47] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:47] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:10:48] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUROC:  0.8280680063129452
Avg Precision:  0.898236086105572


# Random Forest

In [22]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
C=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,1000,10000000]
bestC=0
bestValue=-1
cv=5
for c in C:
    avgP=0
    avgAUROC=0
    skf = StratifiedKFold(n_splits=cv,random_state=10,shuffle=True )
    for train_index, val_index in skf.split(X.values, y):
        X_train1=X.values[train_index]
        X_val=X.values[val_index]
        y_train1=y[train_index]
        y_val=y[val_index]
        clf=RandomForestClassifier(max_depth=2, random_state=0)
        clf.fit(X_train1,y_train1)
        y_pred_val=clf.predict_proba(X_val)[:,1]
        #print(y_pred_val)
        avgP+=average_precision_score(y_val, y_pred_val)
        avgAUROC+=roc_auc_score(y_val, y_pred_val)
    avgP=avgP/cv
    avgROC=avgAUROC/cv
print('AUROC: ',avgROC)
print('Avg Precision: ',avgP)

AUROC:  0.7683401700761278
Avg Precision:  0.8554235987916018


# LogisticRegression

In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
C=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,1000,10000000]
bestC=0
bestValue=-1
cv=5
for c in C:
    avgP=0
    avgAUROC=0
    skf = StratifiedKFold(n_splits=cv,random_state=10,shuffle=True )
    for train_index, val_index in skf.split(X.values, y):
        X_train1=X.values[train_index]
        X_val=X.values[val_index]
        y_train1=y[train_index]
        y_val=y[val_index]
        clf=LogisticRegression(random_state=0, class_weight='balanced',C=c,penalty='l2')
        clf.fit(X_train1,y_train1)
        y_pred_val=clf.predict_proba(X_val)[:,1]
        #print(y_pred_val)
        avgP+=average_precision_score(y_val, y_pred_val)
        avgAUROC+=roc_auc_score(y_val, y_pred_val)
    avgP=avgP/cv
    avgROC=avgAUROC/cv
print('AUROC: ',avgROC)
print('Avg Precision: ',avgP)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

AUROC:  0.7920789570889084
Avg Precision:  0.8610846985007553


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

# Extra Tree Classifier

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
C=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,1000,10000000]
bestC=0
bestValue=-1
cv=5
for c in C:
    avgP=0
    avgAUROC=0
    skf = StratifiedKFold(n_splits=cv,random_state=10,shuffle=True )
    for train_index, val_index in skf.split(X.values, y):
        X_train1=X.values[train_index]
        X_val=X.values[val_index]
        y_train1=y[train_index]
        y_val=y[val_index]
        clf=ExtraTreesClassifier(ccp_alpha=0.0,class_weight='balanced',n_estimators=100, random_state=100)
        clf.fit(X_train1,y_train1)
        y_pred_val=clf.predict_proba(X_val)[:,1]
        #print(y_pred_val)
        avgP+=average_precision_score(y_val, y_pred_val)
        avgAUROC+=roc_auc_score(y_val, y_pred_val)
    avgP=avgP/cv
    avgROC=avgAUROC/cv
print('AUROC: ',avgROC)
print('Avg Precision: ',avgP)

AUROC:  0.8159700924904374
Avg Precision:  0.8952898267006366


# SVM

In [32]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.svm import LinearSVC
C=[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100,500,1000,1000,10000000]
bestC=0
bestValue=-1
cv=5
for c in C:
    avgP=0
    avgAUROC=0
    skf = StratifiedKFold(n_splits=cv,random_state=10,shuffle=True )
    for train_index, val_index in skf.split(X.values, y):
        X_train1=X.values[train_index]
        X_val=X.values[val_index]
        y_train1=y[train_index]
        y_val=y[val_index]
        clf=LinearSVC(class_weight='balanced',random_state=100),
        clf[0].fit(X_train1,y_train1)
        y_pred_val=clf[0].predict(X_val)
        #print(y_pred_val)
        avgP+=average_precision_score(y_val, y_pred_val)
        avgAUROC+=roc_auc_score(y_val, y_pred_val)
    avgP=avgP/cv
    avgROC=avgAUROC/cv
print('AUROC: ',avgROC)
print('Avg Precision: ',avgP)

C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\skle

C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Anaconda3\lib\site-packages\skle

AUROC:  0.5833228433073634
Avg Precision:  0.713399063520361


C:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [35]:
classif = ['Logistic Regression','SVM','Random Forest','Extra Trees','XGBoost']

classifier_real_network_1 = pd.DataFrame(
    {'Classifier': classif,
     'AUROC': [0.7920789570889084,0.5833228433073634,0.7683401700761278,0.8159700924904374,0.8280680063129452],
     'AvgP': [0.8610846985007553,0.713399063520361,0.8554235987916018,0.8952898267006366,0.898236086105572],
    })

classifier_real_network_1


,Classifier,AUROC,AvgP
0,Logistic Regression,0.792079,0.861085
1,SVM,0.583323,0.713399
2,Random Forest,0.768340,0.855424
3,Extra Trees,0.815970,0.895290
4,XGBoost,0.828068,0.898236


In [37]:
final_result.to_csv("allfcombine.csv")